In [1]:
%matplotlib inline

# Scientific computing
import numpy as np
import pandas as pd
from scipy import interp

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#sns.set_context('paper')

# Machine Learning
# Model
import lightgbm as lgb
# Splitter Functions
from sklearn.model_selection import train_test_split

# Other
import requests
import threading

Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [2]:
first_cycle = True
with pd.HDFStore('../../classification/ris/OUT-classified-merged.h5', mode='r') as in_data:
    data = pd.DataFrame(in_data['GLITCH'].to_numpy().sort(axis=1))
    data = pd.concat((data, pd.DataFrame(in_data['MULTI_GLITCH'].to_numpy().sort(axis=1))))
    data = pd.concat((data, pd.DataFrame(in_data['NO_GLITCH'].to_numpy().sort(axis=1))))
    
    target = np.ones(len(in_data['GLITCH'].to_numpy()))
    target = np.concatenate((target, np.ones(len(in_data['MULTI_GLITCH'].to_numpy()))))
    target = np.concatenate((target, np.zeros(len(in_data['NO_GLITCH'].to_numpy()))))
    #target = pd.DataFrame(target)
    
    #data['target'] = target

In [3]:
first_cycle = True
with pd.HDFStore('../../classification/ris/OUT-classified-merged.h5', mode='r') as in_data:
    for group in ['GLITCH', 'NO_GLITCH']:
        if first_cycle == True:
            data = np.array(in_data[group].to_numpy())
            if group == 'GLITCH':
                target = np.ones(len(data))
            elif group == 'NO_GLITCH':
                target = np.zeros(len(data))
            else:
                print("ERROR.")
            first_cycle = False
        else:
            data = np.concatenate((data, in_data[group].to_numpy()))
            if group == 'GLITCH':
                target = np.concatenate((target, np.ones(len(in_data[group].to_numpy()))))
            elif group == 'NO_GLITCH':
                target = np.concatenate((target, np.zeros(len(in_data[group].to_numpy()))))
            else:
                print("ERROR.")
    data = np.concatenate((data, in_data['MULTI_GLITCH'].to_numpy()))
    target = np.concatenate((target, np.ones(len(in_data['MULTI_GLITCH'].to_numpy()))))

In [4]:
data.sort(axis=1)

In [5]:
data = pd.DataFrame(data)

In [6]:
data['target'] = target

In [7]:
train, test = train_test_split(data, test_size=0.2, stratified=True)

TypeError: Invalid parameters passed: {'stratified': True}

In [7]:
y_train = train['target']
y_test = test['target']
X_train = train.drop('target', axis=1)
X_test = test.drop('target', axis=1)

In [8]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [9]:
params = {
    'num_leaves': 52,
    'metric': ['xentropy', 'l1'],
    'verbose': -1,
    'early_stopping_rounds': 50,
    'learning_rate': 0.0481
}

In [10]:
evals_result = {}  # to record eval results for plotting
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=[lgb_train, lgb_test],
                evals_result=evals_result,
                verbose_eval=10)

/opt/miniconda3/envs/idp/lib/python3.6/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds.
[10]	training's l1: 0.393418	training's xentropy: 0.504425	valid_1's l1: 0.443157	valid_1's xentropy: 0.597642
[20]	training's l1: 0.31133	training's xentropy: 0.380326	valid_1's l1: 0.396619	valid_1's xentropy: 0.531568
[30]	training's l1: 0.248048	training's xentropy: 0.293117	valid_1's l1: 0.360697	valid_1's xentropy: 0.484452
[40]	training's l1: 0.198756	training's xentropy: 0.228815	valid_1's l1: 0.333823	valid_1's xentropy: 0.447977
[50]	training's l1: 0.162952	training's xentropy: 0.182404	valid_1's l1: 0.317318	valid_1's xentropy: 0.423791
[60]	training's l1: 0.136022	training's xentropy: 0.147412	valid_1's l1: 0.30798	valid_1's xentropy: 0.408028
[70]	training's l1: 0.115851	training's xentropy: 0.121505	valid_1's l1: 0.300253	valid_1's xentropy: 0.394839
[80]	training's l1: 0.0990155	training's xentropy: 0.100104	valid_1's l1: 0.294926	valid_1's xentropy: 0.386119
[90]	training's l1: 0.0852536	training's xentropy: 

In [11]:
def render_metric(metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    plt.show()

try:
    # To enable interactive mode you should install ipywidgets
    # https://github.com/jupyter-widgets/ipywidgets
    from ipywidgets import interact, SelectMultiple
    INTERACTIVE = True
except ImportError:
    INTERACTIVE = False

if INTERACTIVE:
    # create widget to switch between metrics
    interact(render_metric, metric_name=params['metric'])
else:
    render_metric(params['metric'][0])

interactive(children=(Dropdown(description='metric_name', options=('xentropy', 'l1'), value='xentropy'), Outpu…

In [12]:
ris = gbm.predict(X_test)
ris[ris>=0.5] = 1
ris[ris<0.5] = 0

In [13]:
test['ris'] = ris

/opt/miniconda3/envs/idp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
test[test['ris'] == test['target']].shape[0]/test.shape[0]

0.885

In [166]:
boo

629    False
585     True
25      True
321     True
418    False
       ...  
94      True
8      False
588    False
231     True
326    False
Length: 400, dtype: bool

In [ ]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_train_aug, y_train_aug = X_train, y_train
X_train_aug = np.concatenate((X_train_aug, -X_train))
y_train_aug = np.concatenate((y_train_aug, y_train))
for j in range(1,100):
    X_train_aug = np.concatenate((X_train_aug, np.roll(X_train, j, axis=1)))
    X_train_aug = np.concatenate((X_train_aug, -np.roll(X_train, j, axis=1)))
    y_train_aug = np.concatenate((y_train_aug, y_train))
    y_train_aug = np.concatenate((y_train_aug, y_train))
X_train
# Apply data augmentation on testing data
X_test_aug, y_test_aug = X_test, y_test
X_test_aug = np.concatenate((X_test_aug, -X_test))
y_test_aug = np.concatenate((y_test_aug, y_test))
for j in range(1,100):
    X_test_aug = np.concatenate((X_test_aug, np.roll(X_test, j, axis=1)))
    X_test_aug = np.concatenate((X_test_aug, -np.roll(X_test, j, axis=1)))
    y_test_aug = np.concatenate((y_test_aug, y_test))
    y_test_aug = np.concatenate((y_test_aug, y_test))

In [171]:
from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin

In [175]:
N_FOLDS = 10


def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    cv_results = lgb.cv(params, lgb_train, nfold = n_folds, num_boost_round = 10000, early_stopping_rounds = 100, metrics = 'auc', seed = 50)
  
    # Extract the best score
    best_score = max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'num_leaves': 2+hp.randint('num_leaves', 150),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),

    
}

tpe_algorithm = tpe.suggest
bayes_trials = Trials()
MAX_EVALS=5

In [176]:
#best=[]

lgb_train = lgb.Dataset(X_train, y_train)
tpe_algorithm = tpe.suggest
bayes_trials = Trials()

b = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)
print(b)
#best.append(b)

100%|██████████| 5/5 [01:48<00:00, 21.77s/it, best loss: 0.0001258009063918042]
{'learning_rate': 0.04815461602168385, 'num_leaves': 53}
